### Add Directories/ Load Files

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from utils import find_folders
import mne
from scipy.stats import mannwhitneyu

In [ ]:
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

In [ ]:
path_raw_accel = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Accelerometer\\subcohort_noENTRAINMENT\\raw'

Handle TMSI Accelerometer Data

In [ ]:
# RUN THIS CODE
import tmsi_poly5reader

poly5_name = 'Sub020_12mfu_M1S0_BStr_RSTN_FTRamp-20220509T133001.DATA.Poly5'

# LOAD FILE
accel = tmsi_poly5reader.Poly5Reader(
    os.path.join(path_raw_accel,
                 poly5_name)
)

raw_acc = accel.read_data_MNE()
dn_accel = raw_acc.resample(250, npad="auto")


In [ ]:
dn_accel.ch_names


In [ ]:
my_dn_accel = dn_accel.get_data(picks=[3,4,5])
my_dn_accel.shape
plt.plot(my_dn_accel[2,:])
plt.show()


In [ ]:
np.savetxt(os.path.join(
    path_raw_accel,
    'Sub020_M1_250Hz_LHAND.csv'
), my_dn_accel, delimiter = ',')

In [ ]:
accel_blocks_df = pd.read_excel(os.path.join(
    project_path,
    'results',
    'accelerometer',
    'Accel_Blocks_NoEntrainment.xlsx'
))

accel_blocks_df.head(10)

In [ ]:
features_path = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Accelerometer\\subcohort_noENTRAINMENT\\retap_results\\features'
blocks_path = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Accelerometer\\subcohort_noENTRAINMENT\\retap_results\\extracted_tapblocks'

In [ ]:
imp_features = ['freq', 'mean_raise_velocity', 'coefVar_raise_velocity', 'trace_RMSn', 'coefVar_intraTapInt', 'slope_intraTapInt','trace_entropy','jerkiness_trace', 'mean_tapRMS',
                'coefVar_tapRMS', 'mean_impactRMS', 'coefVar_impactRMS', 'slope_impactRMS', 'coefVar_tap_entropy', 'slope_tap_entropy']

In [ ]:
all_subs = accel_blocks_df['Percept_ID'].unique()

for sub in all_subs:
    print(sub)

In [ ]:
conditions = [1,4]

for sub in all_subs:

    this_sub = sub

    for cond in conditions:

        this_cond = cond

        these_blocks = accel_blocks_df.loc[(accel_blocks_df['Percept_ID'] == this_sub) & (accel_blocks_df['Cond'] == this_cond), 'Blocks']
        block_strings = ['block' + str(num) for num in these_blocks]
        print(block_strings)

        # Filter the json files based on conditions
        jsons_to_import = []
        for filename in os.listdir(features_path):
            if filename.endswith('.json') and this_sub in filename:
                for block_str in block_strings:
                    if filename.endswith(f'{block_str}.json'):
                        jsons_to_import.append(filename)

        print(jsons_to_import)

        # Filter the csv files based on conditions
        csvs_to_import = []
        for filename in os.listdir(blocks_path):
            if filename.endswith('.csv') and this_sub in filename:
                for block_str in block_strings:
                    if filename.endswith(f'{block_str}_250Hz.csv'):
                        csvs_to_import.append(filename)

        print(csvs_to_import)

        # Read the selected JSON files

        combined_dict = []
        for filename in jsons_to_import:
            file_path = os.path.join(features_path, filename)
            with open(file_path) as f:
                this_block_feat = json.load(f)
                matching_keys = set(imp_features) & set(this_block_feat.keys())
                combined_dict.append({key: this_block_feat[key] for key in matching_keys})

        #SAVE IT
        suptitle = str(this_sub) + ' - Condition ' + str(this_cond)

        file_name = "".join(suptitle.split()) + '_features.json'

        #PLOT IT
        %matplotlib qt
        fig, axs = plt.subplots(len(csvs_to_import),1, figsize = (18,10))

        suptitle = str(this_sub) + ' - Condition ' + str(this_cond)
        fig.suptitle(suptitle, fontsize=14, fontweight='bold')

        rounded_list = [
            {key: round(value, 2) for key, value in dictionary.items()}
            for dictionary in combined_dict
        ]

        for num, file in enumerate(csvs_to_import):
            this_block_csv = pd.read_csv(os.path.join(blocks_path,file))
            axs[num].plot(this_block_csv)
            axs[num].set_ylabel('Acceleration [g] - ' + str(block_strings[num]))
            axs[num].set_title(rounded_list[num])

            plt.savefig(os.path.join(
            project_path,
            'results',
            'accelerometer',
            'inspections',
            'group_NO_Entrainment',
            "".join(suptitle.split()),
            ), dpi = 250
            )
                
            # Combine the file name with the current directory to create the file path
            file_path = os.path.join(
                project_path,
                'results',
                'accelerometer',
                'inspections',
                'group_NO_Entrainment')

            # Serializing json
            json_object = json.dumps(combined_dict, indent=4)
            
            # Writing to sample.json
            with open(os.path.join(file_path,file_name), "w") as outfile:
                outfile.write(json_object)

In [ ]:
%matplotlib qt
fig, axs = plt.subplots(len(csvs_to_import),1, figsize = (18,10))

suptitle = str(this_sub) + ' - Condition ' + str(this_cond)
fig.suptitle(suptitle, fontsize=14, fontweight='bold')

rounded_list = [
    {key: round(value, 2) for key, value in dictionary.items()}
    for dictionary in combined_dict
]

for num, file in enumerate(csvs_to_import):
    this_block_csv = pd.read_csv(os.path.join(blocks_path,file))
    axs[num].plot(this_block_csv)
    axs[num].set_ylabel('Acceleration [g] - ' + str(block_strings[num]))
    axs[num].set_title(rounded_list[num])
    

In [ ]:
plt.savefig(os.path.join(
    project_path,
    'results',
    'accelerometer',
    'inspections',
    "".join(suptitle.split()),
), dpi = 250
)

### Average all blocks for each condition within subject

In [ ]:
json_dir = os.path.join(
    project_path,
    'results', 'accelerometer', 'inspections', 'group_NO_Entrainment', 'Condition_4'
)

all_features_cond1 = pd.DataFrame()

# Loop through the JSON files in the directory
for filename in os.listdir(json_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(json_dir, filename)
        percept_id = filename[:6]

        # Load the JSON file
        with open(file_path, 'r') as f:
            json_data = json.load(f)
        
        # Initialize a dictionary to store the accumulated sums and counts for each key
        summed_values = {}
        count_values = {}
        
        # Iterate through the dictionaries in the JSON data
        for data_dict in json_data:
            # Accumulate the sums and counts for each key
            for key, value in data_dict.items():
                if key not in summed_values:
                    summed_values[key] = value
                    count_values[key] = 1
                else:
                    summed_values[key] += value
                    count_values[key] += 1
        
        # Calculate the averages for each key
        #averaged_values = {key: summed_values[key] / count_values[key] for key in summed_values}
        averaged_values = {key: summed_values[key] for key in summed_values}
        averaged_values['Percept_ID'] = percept_id
        
        # Append the averaged values to the dataframe
        all_features_cond1 = all_features_cond1.append(averaged_values, ignore_index=True)




In [ ]:
# Print the resulting dataframe
all_features_cond1 = all_features_cond1.reindex(columns=['Percept_ID'] + list(all_features_cond1.columns[:-1]))
print(all_features_cond1)

In [ ]:
# Save the dataframe as JSON
json_file_path = os.path.join(json_dir, 'all_featuresNOENTR_cond4.json')
all_features_cond1.to_json(json_file_path, orient='records')

### Add all blocks together without averaging them

In [ ]:
json_dir = os.path.join(project_path, 'results', 'accelerometer', 'MotorPerf_Comps')

filenames = os.listdir(json_dir)
json_filenames = [filename for filename in filenames if not filename.startswith('.DS_Store')]

data_frames = []
for filename in json_filenames:
    file_path = os.path.join(json_dir, filename)
    with open(file_path, 'r', encoding='utf-8-sig') as f:
        data = json.load(f)
        df = pd.DataFrame(data)
        df.insert(0, 'Percept_ID',filename[:6])   # Extract filename without extension
        data_frames.append(df)

# Concatenate all DataFrames into a single big DataFrame
big_dataframe = pd.concat(data_frames, ignore_index=True)

In [ ]:
json_file_path = os.path.join(json_dir, 'all_featureswithENTR_cond4_ALLBLOCKS.json')
big_dataframe.to_json(json_file_path, orient='records')

### Plot all averaged values in three conditions

In [ ]:
'''json_dir = os.path.join(
    project_path,
    'results', 'accelerometer', 'inspections\\'
)'''

json_dir = os.path.join(project_path, 'results', 'accelerometer', 'MotorPerf_Comps')

feat_cond_1 = pd.read_json(os.path.join(json_dir,'all_featuresNOENTR_cond4_ALLBLOCKS.json'))
feat_cond_2 = pd.read_json(os.path.join(json_dir,'all_featureswithENTR_cond4_ALLBLOCKS.json'))

fts_of_interest = ['Percept_ID',"coefVar_intraTapInt", "trace_RMSn", "coefVar_impactRMS", "mean_raise_velocity"]
feat_cond_1 = feat_cond_1[fts_of_interest]
feat_cond_2 = feat_cond_2[fts_of_interest]

#feat_cond_4 = pd.read_json(str(json_dir) + 'all_features_cond4.json')


In [ ]:
# Get the column values as NumPy arrays
column1_values = feat_cond_1['Percept_ID'].values
column2_values = feat_cond_2['Percept_ID'].values
#column3_values = feat_cond_4['Percept_ID'].values

# Find the similar values between the three columns
similar_values = np.intersect1d(column1_values, np.intersect1d(column2_values, column3_values))
print(similar_values)

# Keep only the rows with similar values in each dataframe
feat_cond_1_filtered = feat_cond_1[feat_cond_1['Percept_ID'].isin(similar_values)]
feat_cond_2_filtered = feat_cond_2[feat_cond_2['Percept_ID'].isin(similar_values)]
#feat_cond_4_filtered = feat_cond_4[feat_cond_4['Percept_ID'].isin(similar_values)]

In [ ]:
feat_cond_1['Cohort'] = 1
feat_cond_2['Cohort'] = 2
lme_dataframe = pd.concat([feat_cond_1,feat_cond_2], ignore_index=True)
lme_dataframe.to_excel('LME_MotorPerformance.xlsx')

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
lme_dataframe = pd.read_excel(os.path.join(project_path, 'results', 'accelerometer', 'MotorPerf_Comps','LME_MotorPerformance.xlsx'))
lme_dataframe

In [ ]:
md = smf.mixedlm("mean_raise_velocity ~ Stim", lme_dataframe, groups=lme_dataframe["Percept_ID"])
mdf = md.fit()
print(mdf.summary())

In [ ]:
%matplotlib qt
#import mlxtend
from scipy.stats import mannwhitneyu

# Get the column names excluding 'Percept_ID'
columns = [col for col in feat_cond_1.columns if col != 'Percept_ID']

#fig, axes = plt.subplots(1, len(columns), figsize=(25, 5))  # Adjust figsize as needed

fig, axes = plt.subplots(1, 4, figsize=(10, 3))
axes = axes.flatten()
# Iterate over each column and create a boxplot for each dataframe
for i, column in enumerate(columns):
    # Get the data for the column from each dataframe
    data1 = feat_cond_1[column]
    data2 = feat_cond_2[column]
    #data3 = feat_cond_4[column]

    #p_value = mlxtend.permutation_test(data1.values, data2.values, paired=False, method="approximate", seed=0, num_rounds=100000)

    #p_value_round = np.round(p_value, decimals = 6)

    statistic, p_value = mannwhitneyu(data1, data2)
    p_value_round = np.round(p_value, decimals = 6)


     # Create a boxplot for the column in the corresponding subplot
    axes[i].boxplot([data1, data2], widths = 0.5, showfliers=False)
    axes[i].scatter([1]*len(data1), data1, color='dimgrey', s =20)
    axes[i].scatter([2]*len(data2), data2, color='darkred', s=20)
    #axes[i].scatter([3]*len(data3), data3, color='green')
    axes[i].set_title(column)


    # Create a boxplot for the column in the corresponding subplot
    axes[i].set_title(column + ':' + str(p_value_round))

# Add a legend at the bottom of the plot
#legend_labels = ['M1S0', 'M1S1:PreSubharmonic', 'M1S1:Subh-Highest Amp']
#legend_markers = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color) for color in ['red', 'blue', 'green']]

legend_labels = ['GroupNO', 'GroupWith']
legend_markers = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color) for color in ['dimgrey', '#DDCC77']]

fig.legend(legend_markers, legend_labels, loc='lower center', ncol=3)

# Adjust spacing between subplots and legend
fig.tight_layout(rect=[0, 0.1, 1, 1])

In [ ]:
plt.savefig(os.path.join(
    json_dir, 'All_Blocks_GROUPSCOMPARISONS_WITH_WITHOUT_Wilcoxon'
), dpi = 400
)

plt.savefig(os.path.join(
    json_dir, 'All_Blocks_GROUPSCOMPARISONS_WITH_WITHOUT_Wilcoxon.pdf'
))

## Compare ephys during movement and subharmonic

#### Fix Package Loss in Json Files

In [ ]:
import json
file_Name = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PERCEPT_MOTHERFOLDER\\RawFiles\\sub-028\\12MFU\\Dyskinesia\\Report_Json_Session_Report_20220825T131757.json'

with open(file_Name, 'r') as file:
    # Load the JSON data
    data = json.load(file)

In [ ]:
len(data['BrainSenseTimeDomain'][0]['TimeDomainData'])/250/60

In [ ]:
import re
ticks = data['BrainSenseTimeDomain'][0]['TicksInMses']
ticks_n = np.array(re.findall(r'\d+', ticks), dtype=int)
print(ticks_n)
'''diff = np.diff(ticks_n)
indices = np.where(diff > 250)[0] + 1
print(indices)'''


In [ ]:
ticks_n.shape

#### Synchronize Accel data

In [ ]:
fif_path = os.path.join(project_path, 'data', 'Fifs', 'with_med_FTG')
accel_path = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Accelerometer\\raw_csv_all'
accel_fileName = 'Sub029_M1_250Hz_LHAND.csv'

dn_accel = pd.read_csv(os.path.join(
    accel_path,
    accel_fileName
), header = None
)

Draw = mne.io.read_raw_fif(os.path.join(fif_path,'Sub029_FIF.fif'))

In [ ]:
%matplotlib qt

time_points = np.arange(dn_accel.shape[1]) / 250

for i in range(3):
    plt.plot(time_points, dn_accel.iloc[i,:])

# Show the modified figure
plt.show()


In [ ]:
print(f'Number of Samples of Accel data = {dn_accel.shape[1]}')
print(f'Number of Samples of Raw data = {Draw.n_times}')

In [ ]:
num_samples_start = 23.767 *250
num_samples_start

In [ ]:
dn_accel_array = dn_accel.values

num_samples_start = 23.767 *250
data_trimmed = dn_accel_array[:, 5942:]
data_trimmed.shape

#data_trimmed = data_trimmed[:, :-6781]
data_trimmed.shape


In [ ]:
stim = Draw.get_data(picks = 'STIM_R_145Hz_60')[0,:]
plt.plot(data_trimmed[0,:])
plt.plot(stim)

In [ ]:
np.save(os.path.join(
    accel_path,
    'Sub029_cut_traces.npy'
), data_trimmed)

#### Define Rest vs Movement

In [ ]:
SUBID = 'Sub025'

#define your paths
fif_path = os.path.join(project_path, 'data', 'Fifs', 'without_med_FTG')
FFTs_path = os.path.join(project_path, 'data', 'FFTs', 'without_med_FTG')
cut_traces_path = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Accelerometer\\raw_csv_all\\Synced_traces'

#Import your data
accel = np.load(os.path.join(cut_traces_path, 'Sub025_cut_traces.npy'))

#accel = np.genfromtxt(os.path.join(cut_traces_path, 'Sub033_cut_traces.csv'), delimiter=',')

Draw = mne.io.read_raw_fif(os.path.join(fif_path,'Sub025_Connected_.fif'))
D = np.load(os.path.join(FFTs_path, f'{SUBID}_FFT_Connected.npy'))

Draw.ch_names

In [ ]:
%matplotlib qt
time_points1 = np.arange(accel.shape[1]) / 250
time_points2 = np.arange(Draw.n_times) / 250
stim = Draw.get_data(picks = 'STIM_R_125Hz_60us')[0]

plt.plot(time_points1, accel[2,:])
plt.plot(time_points2, stim)

In [ ]:
epoch_df = pd.read_excel(os.path.join(project_path, 'documents', 'FTG_Global.xlsx'), sheet_name = 'Rec_Description')
epoch_df.iloc[:, [1] + list(range(11, 20))]

In [ ]:
row = 7

rest_on = epoch_df.loc[row,'Rest_on']
rest_off = epoch_df.loc[row,'Rest_off']
rest_dur = rest_off - rest_on

block1_on = epoch_df.loc[row,'Block1_on']
block1_off = epoch_df.loc[row,'Block1_off']
block1_dur = block1_off - block1_on

block2_on = epoch_df.loc[row,'Block2_on']
block2_off = epoch_df.loc[row,'Block2_off']
block2_dur = block2_off - block2_on

block3_on = epoch_df.loc[row,'Block3_on']
block3_off = epoch_df.loc[row,'Block3_off']
block3_dur = block3_off - block3_on


In [ ]:
import matplotlib.patches as patches
%matplotlib qt

time_points = np.arange(accel.shape[1]) / 250
fig, ax = plt.subplots()
plt.plot(time_points1, accel[2,:])
plt.plot(time_points2, stim)
ymin, ymax = plt.ylim()

rect1 = patches.Rectangle((rest_on, ymin), rest_dur, ymax, linewidth=1, facecolor='red', alpha = 0.3)
rect2 = patches.Rectangle((block1_on, ymin), block1_dur, ymax, linewidth=1, facecolor='turquoise', alpha = 0.3)
rect3 = patches.Rectangle((block2_on, ymin), block2_dur, ymax, linewidth=1, facecolor='turquoise', alpha = 0.3)
rect4 = patches.Rectangle((block3_on, ymin), block3_dur, ymax, linewidth=1, facecolor='turquoise', alpha = 0.3)

plt.gca().add_patch(rect1)
plt.gca().add_patch(rect2)
plt.gca().add_patch(rect3)
plt.gca().add_patch(rect4)

plt.xlabel('Time [sec]')
plt.ylabel('Acceleration')

plt.title(f'{SUBID}')

plt.show()

In [ ]:
plt.savefig(os.path.join(
    project_path,
    'figures',
    'movement_modul', f'{SUBID}_Epochs.jpg'),
    dpi=200)

In [ ]:
SIDE = 1

rest_ps = np.nanmean(D[SIDE, :, int(rest_on):int(rest_off)], 1)

tap1_ps = np.nanmean(D[SIDE, :, int(block1_on):int(block1_off)], 1)
tap2_ps = np.nanmean(D[SIDE, :, int(block2_on):int(block2_off)], 1)
tap3_ps = np.nanmean(D[SIDE, :, int(block3_on):int(block3_off)], 1)

In [ ]:
plt.plot(np.arange(1,127), rest_ps, label = 'Rest', lw = 2, linestyle = 'dashed')
plt.plot(np.arange(1,127), tap1_ps, label = 'TapBlock1', lw = 2)
plt.plot(np.arange(1,127), tap2_ps, label = 'TapBlock2', lw = 2)
plt.plot(np.arange(1,127), tap3_ps, label = 'TapBlock3', lw = 2)

plt.legend()
plt.xlim(40,90)
plt.ylim(0,0.1)
plt.xlabel('Frequency [Hz]')
plt.ylabel('LFP Power')

plt.title(f'{SUBID}')


In [ ]:
plt.savefig(os.path.join(
    project_path,
    'figures',
    'movement_modul', f'{SUBID}_PowerSpectra.jpg'),
    dpi=200)

In [ ]:
tap_array = np.array([tap1_ps, tap2_ps, tap3_ps])
tap_array.shape

In [ ]:
np.save(os.path.join(project_path, 'data','movement_modul', f'{SUBID}_RestPS.npy'), rest_ps)
np.save(os.path.join(project_path, 'data','movement_modul', f'{SUBID}_TapPS.npy'), tap_array)

### Average all taps and plot

In [ ]:
import glob
data_dir = os.path.join(project_path, 'data', 'movement_modul')

file_list = glob.glob(data_dir + '*TapPS.npy')

# Initialize an empty list to store the arrays
for file_path in file_list:
    array = np.load(file_path)
    print(array)
    break
    averaged_array = np.mean(array, axis=0)
    print(averaged_array.shape)
    
    name = file_path.split('\\')[-1].split('_')[0]
    print(name)

    np.save(os.path.join(
        data_dir, f'{name}_AVG_TAP.npy'
    ), averaged_array)

In [ ]:
%matplotlib qt
subject_names = set()
data_dir = os.path.join(project_path, 'data', 'movement_modul')
# Collect subject names
for filename in os.listdir(data_dir):
    if filename.endswith(".npy"):
        subject_name = filename.split("_")[0]
        subject_names.add(subject_name)


all_rest = []
all_tap = []

# Load and plot matching pairs
for subject_name in sorted(subject_names):
    tap_file = os.path.join(data_dir, f"{subject_name}_AVG_TAP.npy")
    rest_file = os.path.join(data_dir, f"{subject_name}_RestPs.npy")

    if os.path.isfile(tap_file) and os.path.isfile(rest_file):
        tap_data = np.load(tap_file)
        rest_data = np.load(rest_file)
        print(subject_name)
        all_rest.append(rest_data)
        all_tap.append(tap_data)
        # Plotting code
        plt.figure()
        plt.plot(tap_data, label="Tap")
        plt.plot(rest_data, label="Rest")
        plt.title(f"Subject: {subject_name}")
        plt.ylim(0,0.4)
        plt.xlim(50,70)
        plt.legend()
        plt.show()


In [ ]:
all_rest_df = np.transpose(np.array(all_rest))
all_rest_df = pd.DataFrame(all_rest_df)
all_rest_df.columns = (sorted(subject_names))
all_rest_df

In [ ]:
all_tap_df = np.transpose(np.array(all_tap))
all_tap_df = pd.DataFrame(all_tap_df)
all_tap_df.columns = (sorted(subject_names))
all_tap_df

In [ ]:
data_dir = os.path.join(project_path, 'data', 'movement_modul\\')
all_rest_df = pd.read_excel(os.path.join(data_dir,'AllRest_preTap.xlsx'),index_col=None)
all_tap_df = pd.read_excel(os.path.join(data_dir,'AVG_TAP.xlsx'),index_col=None)
all_tap_df
#all_rest_df.to_excel(os.path.join(data_dir,'AllRest_preTap.xlsx'))
#all_tap_df.to_excel(os.path.join(data_dir,'AVG_TAP.xlsx'))

In [ ]:
colDict = {}
for key in ['Sub005', 'Sub007']:
    colDict[key] = 64
for key in ['Sub021','Sub025','Sub028','Sub033','Sub050','Sub065']:
    colDict[key] = 62
for key in ['Sub029']:
    colDict[key] = 72

max_values_df1 = [0.219394344,
0.028337412,
0.170291695,
0.022434686,
0.025038189,
0.097566164,
0.038175539,
0.0295194,
0.198442069]

max_values_df2 = [0.407060589155108,
0.0348696339338623,
0.230891052766452,
0.0442756168720574,
0.0333978157219401,
0.121905995543275,
0.0451621357921537,
0.0475377818753191,
0.174063343962767
]

statistic, p_value = mannwhitneyu(max_values_df1, max_values_df2)
p_value_round = np.round(p_value, decimals = 6)

print(p_value_round)

In [ ]:
#all_rest_gamma = all_rest_df.iloc[50:81]
#all_tap_gamma = all_tap_df.iloc[50:81]

#rest_smoothed = all_rest_gamma.rolling(window=3, min_periods=1).mean()
#tap_smoothed = all_tap_gamma.rolling(window=3, min_periods=1).mean()

#max_values_df1 = all_rest_gamma.max()
df2_normalized = all_tap_df.div(max_values_df1)
df2_normalized.columns = all_tap_df.columns
df2_normalized.columns 

In [ ]:
# Columns to add NaN rows and shift values
columns_63 = ['Sub021', 'Sub025', 'Sub028','Sub033', 'Sub050', 'Sub065']
columns_73 = ['Sub029']

df2_normalized2 = df2_normalized
# Adding NaN rows and shifting values for specified columns
for column in columns_63:
    # Shift values down by the number of rows
    df2_normalized2[column] = df2_normalized[column].shift(2)

    # Add NaN rows to the beginning of the column
    df2_normalized.loc[:2-1, column] = None

for column1 in columns_73:
    df2_normalized2[column1] = df2_normalized2[column1].shift(-7)
    #df2_normalized2.loc[:10-1, column1] = None


df2_normalized2

In [ ]:
from scipy import stats
mean_tap = np.nanmean(df2_normalized2, axis = 1)
sem_tap = stats.sem(df2_normalized2, axis = 1, nan_policy= 'omit')

plt.plot(np.arange(1,127), mean_tap, color = 'purple', lw = 2)
plt.plot(np.arange(1,127), df2_normalized2, color = 'purple', alpha = 0.18)
plt.fill_between(np.arange(1,127),mean_tap - sem_tap, mean_tap + sem_tap, color = 'purple', alpha=0.3, linestyle = 'dashed', lw = 1.5)
plt.axhline(y = 1, color = 'grey', linestyle = ':', alpha = 0.5)
plt.xlim(50,80)
plt.ylim(0,2.5)

In [ ]:
def plot_columns_with_transparency(dataframe):
    # Line color and transparency
    transparencies = np.linspace(0.3,1,9)

    # Create a figure and axes
    fig, ax = plt.subplots()

    # Plot columns with increasing transparency
    for i, column in enumerate(dataframe.columns):
            dataframe[column].plot(ax=ax, color = 'purple', alpha=transparencies[i], label=f'{column}', lw = 2.5)

    
    # Add labels and title
    ax.set_ylabel('LFP Power [Normalized to Rest]')
    # Add a legend
    ax.legend(loc='best')

    # Show the plot
    plt.show()

# Plot the DataFrame columns
plot_columns_with_transparency(df2_normalized)
plt.axhline(y = 1, color = 'grey', linestyle = ':', alpha = 0.3)
plt.xlim(50,80)
plt.xticks(np.arange(50,85,5), labels = ['-15','-10','-5','1:2\nEntrainment', '+5','+10','+15'])
plt.ylim(0,3)



In [ ]:
plt.savefig(os.path.join(
    project_path,
    'figures',
    'movement_modul', 'Movement_Modulation_All'),
    dpi=200)

plt.savefig(os.path.join(
    project_path,
    'figures',
    'movement_modul', 'Movement_Modulation_All.pdf'))

### Exemplary Patient Movement Modulation

In [ ]:
fft_npy = np.load(os.path.join(
    project_path,
    'data', 'FFTs', 'with_med_FTG',
    'Sub029_FFT.npy'
))

fft_npy.shape

In [ ]:
accel = np.load('S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Accelerometer\\raw_csv_all\\Synced_traces\\Sub029_cut_traces.npy')
accel.shape
import dat_preproc
faccel = dat_preproc.low_highpass_filter(accel, 1, 124)

In [ ]:
stim = np.full((1,323),1.4)

In [ ]:
%matplotlib qt
fig, ax = plt.subplots(2,1)
ax2 = ax[0].twinx()
time_points = np.arange(accel.shape[1]) / 250

ax[0].pcolormesh(fft_npy[1,:,:], cmap = 'viridis', vmin = 0, vmax = 0.8) #vmin = -4, vmax = 7)
ax[0].set_xlim(100,200)
ax[0].set_ylim(60,80)
ax[0].set_xticklabels([])
ax[0].set_yticklabels([])
ax2.plot(np.arange(0, fft_npy.shape[2]), stim[0], color = 'white', linewidth = 2.5, linestyle = 'dotted')
ax2.set_yticks(np.arange(1,3,0.5))
ax2.set_yticklabels([])
ax[1].plot(time_points, faccel[2,:], color = 'grey', alpha = 0.8)
ax[1].set_xlim(100,200)
ax[1].set_xticklabels([])
ax[1].set_yticklabels([])
#plt.

In [ ]:
plt.savefig(os.path.join(
    project_path,
    'figures',
    'movement_modul', 'Sub029_ExemplaryMovementEmpty'),
    dpi=200)

'''plt.savefig(os.path.join(
    project_path,
    'figures',
    'movement_modul', 'Sub029_ExemplaryMovement.jpg'))'''

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import mne  # Import the mne module if not already imported

def save_pdf_without_antialiasing(file_path, fig, dpi=100):
    with PdfPages(file_path) as pdf:
        # Save the figure as a PDF without anti-aliasing
        pdf.savefig(fig, dpi=dpi)

if __name__ == "__main__":
    # Example usage
    # Replace 'figure.pdf' with the desired file path and name for your PDF file
    file_path = "figure.pdf"

    fig, ax = plt.subplots(2, 1, figsize=(8.5, 11))
    bs_data = mne.baseline.rescale(data=fft_npy[1, :, :], times=np.arange(0, fft_npy.shape[2]), baseline=baseline, mode='zscore')

    ax[0].pcolormesh(fft_npy[1, :, :], cmap='viridis', vmin=0, vmax=0.8)
    ax[0].set_xlim(100, 200)
    ax[0].set_ylim(60, 80)
    ax[0].set_ylabel('Frequency [Hz]')

    ax[1].plot(time_points, faccel[2, :], color='grey', alpha=0.8)
    ax[1].set_xlim(100, 200)
    ax[1].set_ylabel('Acceleration [g]')
    plt.xlabel('Time [sec]')

    save_pdf_without_antialiasing(file_path, fig)

    # Show the plot (optional)
    plt.show()
